In [1]:
import numpy as np
import scipy
import scipy.sparse as sp
from scipy.sparse import eye, diags, block_diag, hstack
from scipy.sparse.csgraph import reverse_cuthill_mckee
from scipy.sparse.linalg import gmres, spsolve
from scipy.sparse import eye, diags, kron, csr_matrix
from scipy.sparse.linalg import gmres
# from scipy.sparse.csgraph import symmetrix_degree_order
import importlib
import time
import AllenCahn_Solver as AC_Neu

In [3]:
importlib.reload(AC_Neu)

<module 'AllenCahn_Solver' from '/Users/huynh/Desktop/Python/EnSF/AllenCahn/AllenCahn_Solver.py'>

In [35]:
xa = -0.5 
xb = 0.5
ya = -0.5
yb = 0.5 
T = 10
Nt = 1000
eps = 0.01
Nx = 128
Ny = Nx

N = Nx

hx = (xb - xa) / Nx
hy = (yb - ya) / Ny

h = hx
x = np.arange(xa, xb + hx/2, hx)  # adding hx/2 ensures xb is included
y = np.arange(ya, yb + hy/2, hy)

xmid = (x[:-1]+x[1:])/2
ymid = (y[:-1]+y[1:])/2

dt = T / Nt
TT = np.arange(0, T + dt/2, dt)

S = 2

cons = 0

gamma = 1
## for uniform time step
b0 = 3/(2*dt)
b1 = -1/(2*dt)

# U0 = np.random.uniform(-0.9, 0.9, size=Nx*Ny)

In [37]:
# main diagonal
Main = (-2)*np.ones(N)
Main[0] = -1
Main[-1] = -1
Main *= (1/h**2)

# super- and sub-diagonals
Up =  (1/(h**2))*np.ones(N-1)
Low = (1/(h**2))*np.ones(N-1)

# assemble
G = diags(
    diagonals=[Main, Up, Low],
    offsets=[0, 1, -1],
    shape=(N, N),
    format='csr'
)
# G = np.diag(Main)       \
#   + np.diag(Up,  k= 1) \
#   + np.diag(Low,  k=-1)

I = eye(N, format='csr')
Dh = kron(I, G, format='csr') + kron(G, I, format='csr')

In [9]:
def local_indices(p, Nx, Ny, r):
    """
    Return the flattened indices of a square neighborhood of radius r
    around grid point p on an Nx-by-Ny grid (0-based indexing).

    Parameters:
      p  : integer in [0, Nx*Ny)
      Nx : number of rows
      Ny : number of columns
      r  : localization radius (in grid cells)

    Returns:
      flat_inds : 1D array of length up to (2r+1)^2, listing all
                  0-based flattened indices within the neighborhood.
    """
    # Convert flat index p -> (i,j)
    i = p // Ny
    j = p % Ny
    # Row range [i-r, i+r], clipped to [0, Nx)
    rows = np.arange(max(0, i-r), min(Nx, i+r+1))
    # Col range [j-r, j+r], clipped to [0, Ny)
    cols = np.arange(max(0, j-r), min(Ny, j+r+1))
    # Form Cartesian product
    Ii, Jj = np.meshgrid(rows, cols, indexing='ij')
    flat_inds = (Ii * Ny + Jj).ravel()
    return flat_inds

In [11]:
def nearest_k_obs(i, grid, obs_xy, scalar_idx, K):
    
    if i == scalar_idx:
        return np.array([scalar_idx], dtype=int)
    
    d2 = ((obs_xy - grid[i])**2).sum(axis=1)
    idx = np.argsort(d2)[:K]
    return idx

In [13]:
def LETKF_local(Xb, y, idx_obs, rho, R, Nx, Ny, grid_xy_total, obs_xy, radius):
    """
    Local LETKF using a square neighborhood in model space (no obs localization).

    Steps 1–2: global means and perturbations
    Step 3  : local model-space selection via `local_indices`
    Steps 4–8: analysis in ensemble space (obs from all dims)

    Inputs:
      Xb     : (m_g, k) forecast ensemble (m_g = Nx*Ny)
      y      : (l_g,)    observations
      H      : (l_g, m_g) observation operator
      R      : (l_g, l_g) observation error covariance
      Nx, Ny : grid dimensions
      radius : localization radius in grid cells

    Returns:
      Xa : (m_g, k) analysis ensemble
    """
    m_g, k = Xb.shape

    # Step 1 & 2: global means & perturbations
    Xb_mean = Xb.mean(axis=1, keepdims=True)      # (m_g, 1)
    Xb_pert = Xb - Xb_mean                        # (m_g, k)
    Yb_raw  = np.arctan(Xb[idx_obs, :])          # (l_g, k)                
    Yb_mean = Yb_raw.mean(axis=1, keepdims=True)  # (l_g, 1)
    Yb_pert = Yb_raw - Yb_mean                    # (l_g, k)

    Xa = np.zeros_like(Xb)

    # Precompute obs-space inverse
#     Rinv = np.linalg.inv(R)

    # Loop over each grid point for local analysis
    for p in range(m_g):
        # Step 3: pick local model-space indices
        loc_inds = local_indices(p, Nx, Ny, radius)
        # Local mean & perturbations
        xb_m_loc = Xb_mean[loc_inds, 0]           # (m_loc,)
        Xb_loc   = Xb_pert[loc_inds, :]           # (m_loc, k)

        # Step 4: use all observations (no spatial selection)
        loc_obs = nearest_k_obs(p, grid_xy_total, obs_xy, m_g, 20)
        yb_m_loc = Yb_mean[loc_obs, 0]                  # (l_g,)
        Yb_loc   = Yb_pert[loc_obs, :]                        # (l_g, k)
      
        yo_loc = y[loc_obs, 0] # observation
        R_loc = R[loc_obs]           # now shape (n_obs, l_g)
        R_loc = R_loc[:, loc_obs] 
        # Steps 5–6: ensemble-space analysis covariance
        W = np.linalg.solve(R_loc, Yb_loc)
        C = W.T 

        # Step 7a: analysis perturbation matrix
        M = (k-1)/rho * np.eye(k) + C @ Yb_loc
        w, Q = np.linalg.eigh(M)
        w_inv     = 1.0 / w          # for P^a = Q diag(w_inv) Q^T
        w_inv_sqrt = 1.0 / np.sqrt(w)  # for (P^a)^{1/2}
        
        Pa = (Q * w_inv.reshape(1, -1)) @ Q.T 

        # Step 7b: mean weights
        Wa = np.sqrt(k-1) * (Q * w_inv_sqrt.reshape(-1, 1)) @ Q.T
        wabar = Pa @ (C @ (yo_loc - yb_m_loc))           # (k,)
        Wana  = Wa + wabar.reshape(-1, 1)                  # (k x k)
        
        # Step 8: map back to model space and extract center
        xa_loc = xb_m_loc[:, None] + Xb_loc @ Wana  # (m_loc, k)
        # find center index within the local block
        center_idx = np.where(loc_inds == p)[0][0]
        Xa[p, :] = xa_loc[center_idx, :]

    return Xa

In [15]:
ndim = Nx*Ny
# spa_indices = np.random.permutation(ndim)[:int(1* ndim)] #100 observations

spa_indices = np.random.permutation(ndim)[:int(0.1* ndim)] #10 observations
idx_obs = np.sort(spa_indices)

spa_indices_unobs = np.setdiff1d(np.arange(ndim), idx_obs)
idx_unobs = np.sort(spa_indices_unobs)

In [17]:
idx_obs_rows = idx_obs // (Ny)
idx_obs_cols = idx_obs % (Ny)

In [39]:
data1 = scipy.io.loadmat('RefSol_AllenC_T10_NonConsMob_max0_v3.mat')
U_Ref = data1['RefU']

URef = U_Ref.T

In [41]:
# ntEnSF = 500
ntEnSF = 250
t0 = 0
filtering_steps = ntEnSF
timeTrue = np.linspace(0, 1, Nt+1)
tEnSF = np.linspace(0, 1, filtering_steps+1)
indices_time = np.searchsorted(timeTrue, tEnSF, side='left')

state_timeextract = URef[indices_time, :].copy()

U_EnSF = state_timeextract[:, idx_obs].copy()

dtEnSF = (T - t0) / ntEnSF
obs_sigma = 0.1
SDE_Sigma = 0.01

eps_alpha = 0.05

# ensemble size
ensemble_size = 100
# ensemble_true = 1
# # forward Euler step
# euler_steps = 600
# def g_tau(t):
#     return 1-t

U0_EnSF = np.random.uniform(-0.9, 0.9, size=(ensemble_size, Nx*Ny))

rmse_all = []
obs_save = []
est_save = np.zeros((filtering_steps+1, ndim))
est_save[[0], :] += np.mean(U0_EnSF, axis=0)

In [43]:
Yu, Xu = np.meshgrid(ymid, xmid, indexing='xy')
grid_U = np.stack([Xu.reshape(-1), Yu.reshape(-1)], axis=1)

In [45]:
obs_xy    = grid_U[idx_obs]

In [47]:
U1_EnSF = np.zeros((ensemble_size, Nx*Ny))
noise = 0.1*np.random.randn(Nx*Ny)
for ll in range(ensemble_size):
    U1 = AC_Neu.AllenCahn_Solver_1step_BDF1(U0_EnSF[ll, :], N, dtEnSF, eps, gamma, S, Dh, 6, noise)
    U1_EnSF[ll, :] += U1
    
noiseU = np.sqrt(dtEnSF) * SDE_Sigma * np.random.randn(*U1_EnSF.shape)
U1_EnSF += noiseU

In [49]:
state_scale = U_EnSF[[1], :].copy()
    
indob_scale0 = np.nonzero(((-1e-1 <= state_scale) & (state_scale < -1e-2)) |
                          ((1e-2 <= state_scale) & (state_scale < 1e-1)))[1]

indob_scale1 = np.nonzero(((-1e-2 <= state_scale) & (state_scale < -1e-3)) |
                          ((1e-3 <= state_scale) & (state_scale < 1e-2)))[1]

indob_scale2 = np.nonzero(((-1e-3 <= state_scale) & (state_scale < -1e-4)) |
                          ((1e-4 <= state_scale) & (state_scale < 1e-3)))[1]

indob_scale3 = np.nonzero(((-1e-4 <= state_scale) & (state_scale < -1e-5)) |
                          ((1e-5 <= state_scale) & (state_scale < 1e-4)))[1]

indob_scale4 = np.nonzero(((-1e-5 <= state_scale) & (state_scale < -1e-6)) |
                          ((1e-6 <= state_scale) & (state_scale < 1e-5)))[1]
indob_scale5 = np.nonzero(((-1e-6 <= state_scale) & (state_scale < -1e-7)) |
                          ((1e-7 <= state_scale) & (state_scale < 1e-6)))[1]

indob_scale6 = np.nonzero(((-1e-7 <= state_scale) & (state_scale < -1e-8)) |
                          ((1e-8 <= state_scale) & (state_scale < 1e-7)))[1]

indob_scale7 = np.nonzero(((-1e-8 <= state_scale) & (state_scale < -1e-9)) |
                          ((1e-9 <= state_scale) & (state_scale < 1e-8)))[1]

indob_scale8 = np.nonzero(((-1e-9 <= state_scale) & (state_scale < -1e-10)) |
                          ((1e-10 <= state_scale) & (state_scale < 1e-9)))[1]

indob_scale9 = np.nonzero(((-1e-10 <= state_scale) & (state_scale < -1e-11)) |
                          ((1e-11 <= state_scale) & (state_scale < 1e-10)))[1]

indob_scale10 = np.nonzero(((-1e-11 <= state_scale) & (state_scale < -1e-12)) |
                          ((1e-12 <= state_scale) & (state_scale < 1e-11)))[1]

indob_scale11 = np.nonzero(((-1e-12 <= state_scale) & (state_scale < -1e-13)) |
                          ((1e-13 <= state_scale) & (state_scale < 1e-12)))[1]

indob_scale12 = np.nonzero(((-1e-13 <= state_scale) & (state_scale < -1e-14)) |
                          ((1e-14 <= state_scale) & (state_scale < 1e-13)))[1]

indob_scale13 = np.nonzero(((-1e-14 <= state_scale) & (state_scale < -1e-15)) |
                          ((1e-15 <= state_scale) & (state_scale < 1e-14)))[1]

indob_scale14 = np.nonzero(((-1e-15 <= state_scale) & (state_scale < -1e-16)) |
                          ((1e-16 <= state_scale) & (state_scale < 1e-15)))[1]

indob_scale15 = np.nonzero(((-1e-16 <= state_scale) & (state_scale < -1e-17)) |
                          ((1e-17 <= state_scale) & (state_scale < 1e-16)))[1]
indob_scale16 = np.nonzero(((-1e-17 <= state_scale) & (state_scale < 0)) |
                          ((0 <= state_scale) & (state_scale < 1e-17)))[1]

state_scale[:, indob_scale0] *= 1e1
state_scale[:, indob_scale1] *= 1e2
state_scale[:, indob_scale2] *= 1e3
state_scale[:, indob_scale3] *= 1e4
state_scale[:, indob_scale4] *= 1e5
state_scale[:, indob_scale5] *= 1e6
state_scale[:, indob_scale6] *= 1e7
state_scale[:, indob_scale7] *= 1e8
state_scale[:, indob_scale8] *= 1e9
state_scale[:, indob_scale9] *= 1e10
state_scale[:, indob_scale10] *= 1e11
state_scale[:, indob_scale11] *= 1e12
state_scale[:, indob_scale12] *= 1e13
state_scale[:, indob_scale13] *= 1e14
state_scale[:, indob_scale14] *= 1e15
state_scale[:, indob_scale15] *= 1e16
state_scale[:, indob_scale16] *= 1e17

obs = np.arctan(state_scale.copy())
obs += np.random.randn(*U_EnSF[[1], :].shape) * obs_sigma

x0_EnSF = U1_EnSF[:, idx_obs].copy()
     
indx_scale0 = np.argwhere(((-1e-1<=x0_EnSF) & (x0_EnSF<-1e-2)) | ((1e-2<=x0_EnSF) & (x0_EnSF<1e-1)))
indx_scale1 = np.argwhere(((-1e-2<=x0_EnSF) & (x0_EnSF<-1e-3)) | ((1e-3<=x0_EnSF) & (x0_EnSF<1e-2)))
indx_scale2 = np.argwhere(((-1e-3<=x0_EnSF) & (x0_EnSF<-1e-4)) | ((1e-4<=x0_EnSF) & (x0_EnSF<1e-3)))
indx_scale3 = np.argwhere(((-1e-4<=x0_EnSF) & (x0_EnSF<-1e-5)) | ((1e-5<=x0_EnSF) & (x0_EnSF<1e-4)))
indx_scale4 = np.argwhere(((-1e-5<=x0_EnSF) & (x0_EnSF<-1e-6)) | ((1e-6<=x0_EnSF) & (x0_EnSF<1e-5)))
indx_scale5 = np.argwhere(((-1e-6<=x0_EnSF) & (x0_EnSF<-1e-7)) | ((1e-7<=x0_EnSF) & (x0_EnSF<1e-6)))
indx_scale6 = np.argwhere(((-1e-7<=x0_EnSF) & (x0_EnSF<-1e-8)) | ((1e-8<=x0_EnSF) & (x0_EnSF<1e-7)))
indx_scale7 = np.argwhere(((-1e-8<=x0_EnSF) & (x0_EnSF<-1e-9)) | ((1e-9<=x0_EnSF) & (x0_EnSF<1e-8)))
indx_scale8 = np.argwhere(((-1e-9<=x0_EnSF) & (x0_EnSF<-1e-10)) | ((1e-10<=x0_EnSF) & (x0_EnSF<1e-9)))
indx_scale9 = np.argwhere(((-1e-10<=x0_EnSF) & (x0_EnSF<-1e-11)) | ((1e-11<=x0_EnSF) & (x0_EnSF<1e-10)))
indx_scale10 = np.argwhere(((-1e-11<=x0_EnSF) & (x0_EnSF<-1e-12)) | ((1e-12<=x0_EnSF) & (x0_EnSF<1e-11)))
indx_scale11 = np.argwhere(((-1e-12<=x0_EnSF) & (x0_EnSF<-1e-13)) | ((1e-13<=x0_EnSF) & (x0_EnSF<1e-12)))
indx_scale12 = np.argwhere(((-1e-13<=x0_EnSF) & (x0_EnSF<-1e-14)) | ((1e-14<=x0_EnSF) & (x0_EnSF<1e-13)))
indx_scale13 = np.argwhere(((-1e-14<=x0_EnSF) & (x0_EnSF<-1e-15)) | ((1e-15<=x0_EnSF) & (x0_EnSF<1e-14)))
indx_scale14 = np.argwhere(((-1e-15<=x0_EnSF) & (x0_EnSF<-1e-16)) | ((1e-16<=x0_EnSF) & (x0_EnSF<1e-15)))
indx_scale15 = np.argwhere(((-1e-16<=x0_EnSF) & (x0_EnSF<-1e-17)) | ((1e-17<=x0_EnSF) & (x0_EnSF<1e-16)))
indx_scale16 = np.argwhere(((-1e-17<=x0_EnSF) & (x0_EnSF<0)) | ((0<=x0_EnSF) & (x0_EnSF<1e-17)))

x0_EnSF[indx_scale0[:, 0], indx_scale0[:, 1]] *= 1e1
x0_EnSF[indx_scale1[:, 0], indx_scale1[:, 1]] *= 1e2
x0_EnSF[indx_scale2[:, 0], indx_scale2[:, 1]] *= 1e3
x0_EnSF[indx_scale3[:, 0], indx_scale3[:, 1]] *= 1e4
x0_EnSF[indx_scale4[:, 0], indx_scale4[:, 1]] *= 1e5
x0_EnSF[indx_scale5[:, 0], indx_scale5[:, 1]] *= 1e6
x0_EnSF[indx_scale6[:, 0], indx_scale6[:, 1]] *= 1e7
x0_EnSF[indx_scale7[:, 0], indx_scale7[:, 1]] *= 1e8
x0_EnSF[indx_scale8[:, 0], indx_scale8[:, 1]] *= 1e9
x0_EnSF[indx_scale9[:, 0], indx_scale9[:, 1]] *= 1e10
x0_EnSF[indx_scale10[:, 0], indx_scale10[:, 1]] *= 1e11
x0_EnSF[indx_scale11[:, 0], indx_scale11[:, 1]] *= 1e12
x0_EnSF[indx_scale12[:, 0], indx_scale12[:, 1]] *= 1e13
x0_EnSF[indx_scale13[:, 0], indx_scale13[:, 1]] *= 1e14
x0_EnSF[indx_scale14[:, 0], indx_scale14[:, 1]] *= 1e15
x0_EnSF[indx_scale15[:, 0], indx_scale15[:, 1]] *= 1e16
x0_EnSF[indx_scale16[:, 0], indx_scale16[:, 1]] *= 1e17


U1_EnSF[:, idx_obs] = x0_EnSF.copy()
R = obs_sigma**2 * np.eye(ndim)

Xb_LETKF = U1_EnSF.copy()
Xb = Xb_LETKF.T
y  = obs.T
#     rho = 2
rho = 3
radius = 4

Xa_LETKF = LETKF_local(Xb, y, idx_obs, rho, R, Nx, Ny, grid_U, obs_xy, radius)

x_scale = Xa_LETKF.T
x0_EnSF = x_scale[:, idx_obs].copy()
x0_EnSF[:, indob_scale0] /= 1e1
x0_EnSF[:, indob_scale1] /= 1e2
x0_EnSF[:, indob_scale2] /= 1e3
x0_EnSF[:, indob_scale3] /= 1e4
x0_EnSF[:, indob_scale4] /= 1e5
x0_EnSF[:, indob_scale5] /= 1e6
x0_EnSF[:, indob_scale6] /= 1e7
x0_EnSF[:, indob_scale7] /= 1e8
x0_EnSF[:, indob_scale8] /= 1e9
x0_EnSF[:, indob_scale9] /= 1e10
x0_EnSF[:, indob_scale10] /= 1e11
x0_EnSF[:, indob_scale11] /= 1e12
x0_EnSF[:, indob_scale12] /= 1e13
x0_EnSF[:, indob_scale13] /= 1e14
x0_EnSF[:, indob_scale14] /= 1e15
x0_EnSF[:, indob_scale15] /= 1e16
x0_EnSF[:, indob_scale16] /= 1e17

x_scale[:, idx_obs] = x0_EnSF.copy()
U1_EnSF = x_scale.copy()
# U1_EnSF[:, idx_obs] = x0_EnSF.copy()
U1_EnSF = np.clip(U1_EnSF, -1, 1)

est_save[[1], :] += np.mean(U1_EnSF, axis=0)

In [51]:
for i in range(1, filtering_steps):
    print(f'step={i}:')
    t1 = time.time()    
    
    state_scale = U_EnSF[[i+1], :].copy()
    
    indob_scale0 = np.nonzero(((-1e-1 <= state_scale) & (state_scale < -1e-2)) |
                              ((1e-2 <= state_scale) & (state_scale < 1e-1)))[1]
    
    indob_scale1 = np.nonzero(((-1e-2 <= state_scale) & (state_scale < -1e-3)) |
                              ((1e-3 <= state_scale) & (state_scale < 1e-2)))[1]
    
    indob_scale2 = np.nonzero(((-1e-3 <= state_scale) & (state_scale < -1e-4)) |
                              ((1e-4 <= state_scale) & (state_scale < 1e-3)))[1]

    indob_scale3 = np.nonzero(((-1e-4 <= state_scale) & (state_scale < -1e-5)) |
                              ((1e-5 <= state_scale) & (state_scale < 1e-4)))[1]

    indob_scale4 = np.nonzero(((-1e-5 <= state_scale) & (state_scale < -1e-6)) |
                              ((1e-6 <= state_scale) & (state_scale < 1e-5)))[1]
    indob_scale5 = np.nonzero(((-1e-6 <= state_scale) & (state_scale < -1e-7)) |
                              ((1e-7 <= state_scale) & (state_scale < 1e-6)))[1]
    
    indob_scale6 = np.nonzero(((-1e-7 <= state_scale) & (state_scale < -1e-8)) |
                              ((1e-8 <= state_scale) & (state_scale < 1e-7)))[1]

    indob_scale7 = np.nonzero(((-1e-8 <= state_scale) & (state_scale < -1e-9)) |
                              ((1e-9 <= state_scale) & (state_scale < 1e-8)))[1]
    
    indob_scale8 = np.nonzero(((-1e-9 <= state_scale) & (state_scale < -1e-10)) |
                              ((1e-10 <= state_scale) & (state_scale < 1e-9)))[1]

    indob_scale9 = np.nonzero(((-1e-10 <= state_scale) & (state_scale < -1e-11)) |
                              ((1e-11 <= state_scale) & (state_scale < 1e-10)))[1]
    
    indob_scale10 = np.nonzero(((-1e-11 <= state_scale) & (state_scale < -1e-12)) |
                              ((1e-12 <= state_scale) & (state_scale < 1e-11)))[1]
    
    indob_scale11 = np.nonzero(((-1e-12 <= state_scale) & (state_scale < -1e-13)) |
                              ((1e-13 <= state_scale) & (state_scale < 1e-12)))[1]
    
    indob_scale12 = np.nonzero(((-1e-13 <= state_scale) & (state_scale < -1e-14)) |
                              ((1e-14 <= state_scale) & (state_scale < 1e-13)))[1]
    
    indob_scale13 = np.nonzero(((-1e-14 <= state_scale) & (state_scale < -1e-15)) |
                              ((1e-15 <= state_scale) & (state_scale < 1e-14)))[1]
    
    indob_scale14 = np.nonzero(((-1e-15 <= state_scale) & (state_scale < -1e-16)) |
                              ((1e-16 <= state_scale) & (state_scale < 1e-15)))[1]
    
    indob_scale15 = np.nonzero(((-1e-16 <= state_scale) & (state_scale < -1e-17)) |
                              ((1e-17 <= state_scale) & (state_scale < 1e-16)))[1]
    indob_scale16 = np.nonzero(((-1e-17 <= state_scale) & (state_scale < 0)) |
                              ((0 <= state_scale) & (state_scale < 1e-17)))[1]
    
    state_scale[:, indob_scale0] *= 1e1
    state_scale[:, indob_scale1] *= 1e2
    state_scale[:, indob_scale2] *= 1e3
    state_scale[:, indob_scale3] *= 1e4
    state_scale[:, indob_scale4] *= 1e5
    state_scale[:, indob_scale5] *= 1e6
    state_scale[:, indob_scale6] *= 1e7
    state_scale[:, indob_scale7] *= 1e8
    state_scale[:, indob_scale8] *= 1e9
    state_scale[:, indob_scale9] *= 1e10
    state_scale[:, indob_scale10] *= 1e11
    state_scale[:, indob_scale11] *= 1e12
    state_scale[:, indob_scale12] *= 1e13
    state_scale[:, indob_scale13] *= 1e14
    state_scale[:, indob_scale14] *= 1e15
    state_scale[:, indob_scale15] *= 1e16
    state_scale[:, indob_scale16] *= 1e17
    
    obs = np.arctan(state_scale.copy())
    obs += np.random.randn(*U_EnSF[[i+1], :].shape) * obs_sigma
       
    x_state = np.zeros((ensemble_size, ndim))

    noise = 0.1*np.random.randn(Nx*Ny) 
    for ll in range(ensemble_size):
        x_state[ll, :] = AC_Neu.AllenCahn_Solver_1step_BDF2(U1_EnSF[ll, :], U0_EnSF[ll, :], N, dtEnSF,\
                                                            eps, gamma, S, Dh, b0, b1, 6, noise)
    
    
    noiseU = np.sqrt(dtEnSF) * SDE_Sigma * np.random.randn(*x_state.shape)
    
    x_state += noiseU
    
    R = obs_sigma**2 * np.eye(ndim)
    
    x0_EnSF = x_state[:, idx_obs].copy()

    indx_scale0 = np.argwhere(((-1e-1<=x0_EnSF) & (x0_EnSF<-1e-2)) | ((1e-2<=x0_EnSF) & (x0_EnSF<1e-1)))
    indx_scale1 = np.argwhere(((-1e-2<=x0_EnSF) & (x0_EnSF<-1e-3)) | ((1e-3<=x0_EnSF) & (x0_EnSF<1e-2)))
    indx_scale2 = np.argwhere(((-1e-3<=x0_EnSF) & (x0_EnSF<-1e-4)) | ((1e-4<=x0_EnSF) & (x0_EnSF<1e-3)))
    indx_scale3 = np.argwhere(((-1e-4<=x0_EnSF) & (x0_EnSF<-1e-5)) | ((1e-5<=x0_EnSF) & (x0_EnSF<1e-4)))
    indx_scale4 = np.argwhere(((-1e-5<=x0_EnSF) & (x0_EnSF<-1e-6)) | ((1e-6<=x0_EnSF) & (x0_EnSF<1e-5)))
    indx_scale5 = np.argwhere(((-1e-6<=x0_EnSF) & (x0_EnSF<-1e-7)) | ((1e-7<=x0_EnSF) & (x0_EnSF<1e-6)))
    indx_scale6 = np.argwhere(((-1e-7<=x0_EnSF) & (x0_EnSF<-1e-8)) | ((1e-8<=x0_EnSF) & (x0_EnSF<1e-7)))
    indx_scale7 = np.argwhere(((-1e-8<=x0_EnSF) & (x0_EnSF<-1e-9)) | ((1e-9<=x0_EnSF) & (x0_EnSF<1e-8)))
    indx_scale8 = np.argwhere(((-1e-9<=x0_EnSF) & (x0_EnSF<-1e-10)) | ((1e-10<=x0_EnSF) & (x0_EnSF<1e-9)))
    indx_scale9 = np.argwhere(((-1e-10<=x0_EnSF) & (x0_EnSF<-1e-11)) | ((1e-11<=x0_EnSF) & (x0_EnSF<1e-10)))
    indx_scale10 = np.argwhere(((-1e-11<=x0_EnSF) & (x0_EnSF<-1e-12)) | ((1e-12<=x0_EnSF) & (x0_EnSF<1e-11)))
    indx_scale11 = np.argwhere(((-1e-12<=x0_EnSF) & (x0_EnSF<-1e-13)) | ((1e-13<=x0_EnSF) & (x0_EnSF<1e-12)))
    indx_scale12 = np.argwhere(((-1e-13<=x0_EnSF) & (x0_EnSF<-1e-14)) | ((1e-14<=x0_EnSF) & (x0_EnSF<1e-13)))
    indx_scale13 = np.argwhere(((-1e-14<=x0_EnSF) & (x0_EnSF<-1e-15)) | ((1e-15<=x0_EnSF) & (x0_EnSF<1e-14)))
    indx_scale14 = np.argwhere(((-1e-15<=x0_EnSF) & (x0_EnSF<-1e-16)) | ((1e-16<=x0_EnSF) & (x0_EnSF<1e-15)))
    indx_scale15 = np.argwhere(((-1e-16<=x0_EnSF) & (x0_EnSF<-1e-17)) | ((1e-17<=x0_EnSF) & (x0_EnSF<1e-16)))
    indx_scale16 = np.argwhere(((-1e-17<=x0_EnSF) & (x0_EnSF<0)) | ((0<=x0_EnSF) & (x0_EnSF<1e-17)))

    x0_EnSF[indx_scale0[:, 0], indx_scale0[:, 1]] *= 1e1
    x0_EnSF[indx_scale1[:, 0], indx_scale1[:, 1]] *= 1e2
    x0_EnSF[indx_scale2[:, 0], indx_scale2[:, 1]] *= 1e3
    x0_EnSF[indx_scale3[:, 0], indx_scale3[:, 1]] *= 1e4
    x0_EnSF[indx_scale4[:, 0], indx_scale4[:, 1]] *= 1e5
    x0_EnSF[indx_scale5[:, 0], indx_scale5[:, 1]] *= 1e6
    x0_EnSF[indx_scale6[:, 0], indx_scale6[:, 1]] *= 1e7
    x0_EnSF[indx_scale7[:, 0], indx_scale7[:, 1]] *= 1e8
    x0_EnSF[indx_scale8[:, 0], indx_scale8[:, 1]] *= 1e9
    x0_EnSF[indx_scale9[:, 0], indx_scale9[:, 1]] *= 1e10
    x0_EnSF[indx_scale10[:, 0], indx_scale10[:, 1]] *= 1e11
    x0_EnSF[indx_scale11[:, 0], indx_scale11[:, 1]] *= 1e12
    x0_EnSF[indx_scale12[:, 0], indx_scale12[:, 1]] *= 1e13
    x0_EnSF[indx_scale13[:, 0], indx_scale13[:, 1]] *= 1e14
    x0_EnSF[indx_scale14[:, 0], indx_scale14[:, 1]] *= 1e15
    x0_EnSF[indx_scale15[:, 0], indx_scale15[:, 1]] *= 1e16
    x0_EnSF[indx_scale16[:, 0], indx_scale16[:, 1]] *= 1e17
    
    x_state[:, idx_obs] = x0_EnSF.copy()
    
    Xb_LETKF = x_state.copy()
    Xb = Xb_LETKF.T
    y  = obs.T
#     rho = 2
    rho = 3
    radius = 4
    Xa_LETKF = LETKF_local(Xb, y, idx_obs, rho, R, Nx, Ny, grid_U, obs_xy, radius)
    
    x_scale = Xa_LETKF.T
    x0_EnSF = x_scale[:, idx_obs].copy()
    
    x0_EnSF[:, indob_scale0] /= 1e1
    x0_EnSF[:, indob_scale1] /= 1e2
    x0_EnSF[:, indob_scale2] /= 1e3
    x0_EnSF[:, indob_scale3] /= 1e4
    x0_EnSF[:, indob_scale4] /= 1e5
    x0_EnSF[:, indob_scale5] /= 1e6
    x0_EnSF[:, indob_scale6] /= 1e7
    x0_EnSF[:, indob_scale7] /= 1e8
    x0_EnSF[:, indob_scale8] /= 1e9
    x0_EnSF[:, indob_scale9] /= 1e10
    x0_EnSF[:, indob_scale10] /= 1e11
    x0_EnSF[:, indob_scale11] /= 1e12
    x0_EnSF[:, indob_scale12] /= 1e13
    x0_EnSF[:, indob_scale13] /= 1e14
    x0_EnSF[:, indob_scale14] /= 1e15
    x0_EnSF[:, indob_scale15] /= 1e16
    x0_EnSF[:, indob_scale16] /= 1e17
    
    x_scale[:, idx_obs] = x0_EnSF.copy()

    x_state = x_scale.copy()
    # x_state[:, idx_obs] = x_scale[:, idx_obs].copy()
    x_state = np.clip(x_state, -1, 1)
    
    U0_EnSF = U1_EnSF.copy()
    U1_EnSF = x_state.copy()
    # print(q_batch)
    est_save[[i+1], :] += np.mean(x_state, axis=0)
#     print(est_save[i+1, -1])
    # get rmse
    rmse_temp = np.sqrt(np.mean((est_save[[i+1], :] - state_timeextract[[i+1], :])**2))

    # get time
    t2 = time.time()
    print(f'\t RMSE = {rmse_temp:.4f}')
    print(f'\t time = {t2 - t1:.4f}')

    # save information
    rmse_all.append(rmse_temp)

    # check divergence
    if rmse_temp > 1000:
        print('diverge!')
        break

step=1:
	 RMSE = 0.3719
	 time = 43.0626
step=2:
	 RMSE = 0.3280
	 time = 39.5190
step=3:
	 RMSE = 0.2982
	 time = 39.3966
step=4:
	 RMSE = 0.2777
	 time = 39.8634
step=5:
	 RMSE = 0.2644
	 time = 39.6511
step=6:
	 RMSE = 0.2534
	 time = 38.8492
step=7:
	 RMSE = 0.2465
	 time = 40.0590
step=8:
	 RMSE = 0.2417
	 time = 39.9031
step=9:
	 RMSE = 0.2383
	 time = 40.6920
step=10:
	 RMSE = 0.2366
	 time = 43.0290
step=11:
	 RMSE = 0.2373
	 time = 38.7059
step=12:
	 RMSE = 0.2351
	 time = 39.1068
step=13:
	 RMSE = 0.2329
	 time = 40.6962
step=14:
	 RMSE = 0.2346
	 time = 41.5744
step=15:
	 RMSE = 0.2384
	 time = 40.1792
step=16:
	 RMSE = 0.2408
	 time = 40.2248
step=17:
	 RMSE = 0.2409
	 time = 40.2747
step=18:
	 RMSE = 0.2430
	 time = 40.5844
step=19:
	 RMSE = 0.2439
	 time = 40.4074
step=20:
	 RMSE = 0.2452
	 time = 45.6766
step=21:
	 RMSE = 0.2459
	 time = 41.0877
step=22:
	 RMSE = 0.2465
	 time = 41.5118
step=23:
	 RMSE = 0.2471
	 time = 40.8691
step=24:
	 RMSE = 0.2491
	 time = 40.0139
s

In [53]:
scipy.io.savemat('ResultLETKF_AllenC_NonConsMob_T10_max0_Nt250_10Obs_v2.mat', {'U_EnSF':est_save, 'rmse': rmse_all})